In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

conda install -c anaconda pandas-datareader


In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader.data as web
import scipy.optimize as opt
import math
import scipy.stats as stats
from scipy.stats import norm

/Users/israelcastillo/opt/anaconda3/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [3]:
def get_closes(tickers, start_date=None, end_date=None, freq=None):
    import pandas as pd
    pd.core.common.is_list_like = pd.api.types.is_list_like
    import pandas_datareader.data as web  
    closes = pd.DataFrame(columns = tickers, index=web.YahooDailyReader(symbols=tickers[0], start=start_date, end=end_date, interval=freq).read().index)
    for ticker in tickers:
        df = web.YahooDailyReader(symbols=ticker, start=start_date, end=end_date, interval=freq).read()
        closes[ticker]=df['Adj Close']
    closes.index_name = 'Date'
    closes = closes.sort_index()
    return closes

In [4]:
# Información
ticker= ['TLSA', 'MSFT', 'AAPL', 'BABA', 'FB', 'JNJ', 'GOOGL', 'DOL', 'CJ.TO', 'CSU.TO', 'KL', 'CX', 
         'AMX', 'FMX', 'IBA'] 
start, end= '2010-04-02', '2020-04-02'

In [5]:
daily_closes = get_closes(ticker, start, end, freq='d') #d define es diario
daily_closes

,TLSA,MSFT,AAPL,BABA,FB,JNJ,GOOGL,DOL,CJ.TO,CSU.TO,KL,CX,AMX,FMX,IBA
Date,,,,,,,,,,,,,,,
2018-11-20,4.175,99.955009,173.947418,145.979996,132.429993,140.636765,1030.449951,42.206230,2.764442,862.753967,18.714960,4.757163,12.857124,83.309097,38.323654
2018-11-21,5.295,101.330849,173.750839,149.410004,134.820007,136.353821,1043.430054,42.653622,2.837190,889.482849,19.239769,4.562197,12.993799,85.394791,39.699379
2018-11-23,4.900,101.291542,169.337769,150.330002,131.729996,136.584274,1030.099976,42.349014,2.664413,884.449707,19.358593,4.630435,12.779025,85.670250,39.561802
2018-11-26,4.190,104.632881,171.627838,156.009995,136.380005,136.622940,1055.939941,42.977257,2.582571,891.877197,19.219965,4.445218,11.978506,81.341484,38.520187
2018-11-27,4.480,105.291321,171.254364,156.460007,135.000000,138.410828,1052.280029,42.824959,2.409793,881.977234,19.021923,4.542701,12.144466,81.607109,39.650238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-27,1.780,149.699997,247.740005,188.589996,156.789993,123.160004,1110.260010,36.070000,0.415000,1207.280029,31.865000,1.850000,12.180000,61.340000,34.110001
2020-03-30,1.860,160.229996,254.809998,191.270004,165.949997,133.009995,1146.310059,36.840000,0.385000,1275.900024,30.530001,1.830000,11.910000,60.500000,33.830002
2020-03-31,2.040,157.710007,254.289993,194.479996,166.800003,131.130005,1161.949951,36.930000,0.450000,1279.020020,29.600000,2.120000,11.780000,60.509998,34.000000


In [6]:
daily_logret = np.log(daily_closes/daily_closes.shift()).dropna()
daily_logret

,TLSA,MSFT,AAPL,BABA,FB,JNJ,GOOGL,DOL,CJ.TO,CSU.TO,KL,CX,AMX,FMX,IBA
Date,,,,,,,,,,,,,,,
2018-11-21,0.237649,0.013671,-0.001131,0.023225,0.017886,-0.030927,0.012518,0.010544,0.025975,0.030511,0.027656,-0.041847,0.010574,0.024727,0.035268
2018-11-23,-0.077528,-0.000388,-0.025727,0.006139,-0.023186,0.001689,-0.012858,-0.007167,-0.062830,-0.005675,0.006157,0.014846,-0.016667,0.003221,-0.003471
2018-11-26,-0.156534,0.032455,0.013433,0.037087,0.034691,0.000283,0.024775,0.014726,-0.031198,0.008363,-0.007187,-0.040822,-0.064691,-0.051849,-0.026682
2018-11-27,0.066922,0.006273,-0.002178,0.002880,-0.010170,0.013001,-0.003472,-0.003550,-0.069245,-0.011162,-0.010357,0.021693,0.013760,0.003260,0.028915
2018-11-28,0.001115,0.036474,0.037732,0.018240,0.012953,0.022234,0.036859,0.014563,-0.015209,0.037164,0.051736,0.054293,0.046337,0.031681,0.018903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-27,-0.054658,-0.041928,-0.042284,-0.035064,-0.040927,-0.027311,-0.046340,-0.033264,-0.023811,-0.038917,-0.027441,-0.072949,-0.068226,-0.036810,-0.050032
2020-03-30,0.043963,0.067977,0.028138,0.014111,0.056779,0.076940,0.031954,0.021123,-0.075035,0.055282,-0.042798,-0.010870,-0.022417,-0.013789,-0.008243
2020-03-31,0.092373,-0.015852,-0.002043,0.016643,0.005109,-0.014235,0.013551,0.002440,0.156004,0.002442,-0.030935,0.147100,-0.010975,0.000165,0.005012


In [7]:
# Podemos resumir en un DataFrame
daily_ret_summary = pd.DataFrame(index=['Media Diaria 2018-2020', 'Volatilidad Diaria 2018-2020'], columns=ticker)
daily_ret_summary.loc['Media Diaria 2018-2020']=daily_logret.mean()
daily_ret_summary.loc['Volatilidad Diaria 2018-2020']=daily_logret.std()
daily_ret_summary 

,TLSA,MSFT,AAPL,BABA,FB,JNJ,GOOGL,DOL,CJ.TO,CSU.TO,KL,CX,AMX,FMX,IBA
Media Diaria 2018-2020,-0.00216129,0.00106306,0.000875661,0.000650264,0.000223055,-0.000305962,-2.71304e-05,-0.00056741,-0.0046305,0.00106735,0.00135529,-0.00280795,-0.000468324,-0.00115708,-0.000796816
Volatilidad Diaria 2018-2020,0.0903793,0.0226907,0.0239196,0.020486,0.0236338,0.0179904,0.0205568,0.015746,0.0573975,0.0195785,0.035095,0.0339714,0.0240607,0.0193776,0.023572


In [8]:
# Resumen en base anual
annual_ret_summary = pd.DataFrame(index=['Media Anual 2016-2019', 'Vol Anual 2016-2019'], columns=ticker)
annual_ret_summary.loc['Media Anual 2016-2019']=daily_logret.mean()*252
annual_ret_summary.loc['Vol Anual 2016-2019']=daily_logret.std()*np.sqrt(252)
annual_ret_summary

,TLSA,MSFT,AAPL,BABA,FB,JNJ,GOOGL,DOL,CJ.TO,CSU.TO,KL,CX,AMX,FMX,IBA
Media Anual 2016-2019,-0.544645,0.267891,0.220666,0.163867,0.0562098,-0.0771024,-0.00683687,-0.142987,-1.16689,0.268971,0.341532,-0.707605,-0.118018,-0.291585,-0.200798
Vol Anual 2016-2019,1.43473,0.360203,0.379712,0.325206,0.375174,0.285589,0.326329,0.249961,0.911158,0.310799,0.557116,0.53928,0.381952,0.30761,0.374194


In [9]:
cov = daily_logret.cov()
cov

,TLSA,MSFT,AAPL,BABA,FB,JNJ,GOOGL,DOL,CJ.TO,CSU.TO,KL,CX,AMX,FMX,IBA
TLSA,0.008168,-0.000109,-0.000049,0.000037,0.000023,-0.000104,-0.000177,-0.000053,-0.000246,-0.000047,-0.000185,0.000152,0.000132,-0.000104,0.000249
MSFT,-0.000109,0.000515,0.000449,0.000298,0.000395,0.000255,0.000384,0.000293,0.000585,0.000242,0.000039,0.000415,0.000317,0.000258,0.000272
AAPL,-0.000049,0.000449,0.000572,0.000338,0.000399,0.000252,0.000380,0.000295,0.000627,0.000228,0.000013,0.000452,0.000337,0.000254,0.000281
BABA,0.000037,0.000298,0.000338,0.000420,0.000304,0.000150,0.000258,0.000205,0.000432,0.000153,-0.000017,0.000336,0.000218,0.000170,0.000210
FB,0.000023,0.000395,0.000399,0.000304,0.000559,0.000192,0.000367,0.000251,0.000499,0.000221,0.000026,0.000356,0.000265,0.000214,0.000221
JNJ,-0.000104,0.000255,0.000252,0.000150,0.000192,0.000324,0.000193,0.000168,0.000331,0.000121,0.000054,0.000245,0.000187,0.000156,0.000156
GOOGL,-0.000177,0.000384,0.000380,0.000258,0.000367,0.000193,0.000423,0.000242,0.000545,0.000197,0.000004,0.000389,0.000272,0.000210,0.000233
DOL,-0.000053,0.000293,0.000295,0.000205,0.000251,0.000168,0.000242,0.000248,0.000571,0.000150,0.000106,0.000344,0.000259,0.000208,0.000228
CJ.TO,-0.000246,0.000585,0.000627,0.000432,0.000499,0.000331,0.000545,0.000571,0.003294,0.000312,0.000273,0.000851,0.000628,0.000597,0.000475
CSU.TO,-0.000047,0.000242,0.000228,0.000153,0.000221,0.000121,0.000197,0.000150,0.000312,0.000383,0.000043,0.000195,0.000154,0.000145,0.000142


In [10]:
corr = daily_logret.corr()
corr

,TLSA,MSFT,AAPL,BABA,FB,JNJ,GOOGL,DOL,CJ.TO,CSU.TO,KL,CX,AMX,FMX,IBA
TLSA,1.000000,-0.053052,-0.022864,0.019910,0.010546,-0.064261,-0.095405,-0.036967,-0.047383,-0.026764,-0.058301,0.049444,0.060814,-0.059359,0.116655
MSFT,-0.053052,1.000000,0.827915,0.640395,0.736738,0.625012,0.822781,0.820708,0.448818,0.545555,0.048731,0.538830,0.581477,0.587382,0.508944
AAPL,-0.022864,0.827915,1.000000,0.689874,0.705527,0.584567,0.772882,0.784538,0.456610,0.486198,0.015742,0.555799,0.586009,0.548117,0.498597
BABA,0.019910,0.640395,0.689874,1.000000,0.627055,0.408032,0.611488,0.635927,0.367528,0.380287,-0.024185,0.482558,0.442061,0.427230,0.434005
FB,0.010546,0.736738,0.705527,0.627055,1.000000,0.450655,0.756033,0.675041,0.368204,0.477833,0.031763,0.442856,0.465279,0.468015,0.397341
JNJ,-0.064261,0.625012,0.584567,0.408032,0.450655,1.000000,0.521436,0.594086,0.320863,0.343968,0.086078,0.401475,0.431132,0.446272,0.368582
GOOGL,-0.095405,0.822781,0.772882,0.611488,0.756033,0.521436,1.000000,0.749134,0.462058,0.489445,0.005185,0.557496,0.549925,0.527782,0.480862
DOL,-0.036967,0.820708,0.784538,0.635927,0.675041,0.594086,0.749134,1.000000,0.631983,0.487429,0.192551,0.642279,0.683080,0.683231,0.615472
CJ.TO,-0.047383,0.448818,0.456610,0.367528,0.368204,0.320863,0.462058,0.631983,1.000000,0.277715,0.135744,0.436596,0.454378,0.536785,0.351106
CSU.TO,-0.026764,0.545555,0.486198,0.380287,0.477833,0.343968,0.489445,0.487429,0.277715,1.000000,0.063110,0.292675,0.326562,0.381474,0.308034
